In [5]:
import os.path
import sys
import numpy as np
import pandas as pd

In [6]:
# Provide paths to the respective folders
script_folder = r'D:\UoK\OneDrive - University of Kentucky\github\Transit_ridership\transit_ridership_decline\Factors and Ridership Data\code'
load_data = r'D:\UoK\OneDrive - University of Kentucky\github\Transit_ridership\transit_ridership_decline\Factors and Ridership Data\Estimation_File'
output_folder = r'D:\UoK\OneDrive - University of Kentucky\github\Transit_ridership\transit_ridership_decline\Factors and Ridership Data\Script Outputs'
folder_path = ''
file_name = ''

In [7]:
os.chdir(load_data)
df = pd.read_csv('estimation_file.csv')

In [8]:
df_cluster_gt = df.groupby(['CLUSTER_GT_NEW_11','Mode','Year']).sum().reset_index()
df_cluster_gt_grouped = df.groupby(['CLUSTER_GT_8_GROUPS','Mode','Year']).sum().reset_index()

df_cluster_apta = df.groupby(['CLUSTER_APTA','Mode','Year']).sum().reset_index()
# df_cluster_apta_grouped = df.groupby(['CLUSTER_APTA_GROUPED','Mode','Year']).sum().reset_index()

os.chdir(os.path.join(output_folder, 'Cluster_wise_summation_files'))

df_cluster_gt.to_csv('CLUSTER_GT_NEW_11.csv')
df_cluster_gt_grouped.to_csv('CLUSTER_GT_8_GROUPS.csv')
df_cluster_apta.to_csv('CLUSTER_APTA.csv')
# df_cluster_apta_grouped.to_csv('CLUSTER_APTA_GROUPED.csv')

,Unnamed: 0,CBSA,Mode,Year,MNAME,PROBLEM_FLAG,NOTE,UPT,VRM,UPT_ADJ,...,dockCt_LAGGED_2,dockCt_LAGGED_3,docklessCt_LAGGED_1,docklessCt_LAGGED_2,docklessCt_LAGGED_3,scooterCt_LAGGED_1,scooterCt_LAGGED_2,scooterCt_LAGGED_3,TNC_FLAG_LAGGED_4,TNC_FLAG_LAGGED_5
0,0,10420,Bus,2002,"Akron, OH Metro Area",NaN,NaN,5.226386e+06,4.026372e+06,5.226386e+06,...,0,0,0,0,0,0,0,0,0,0
1,1,10420,Bus,2003,"Akron, OH Metro Area",NaN,NaN,5.395902e+06,3.976569e+06,5.395902e+06,...,0,0,0,0,0,0,0,0,0,0
2,2,10420,Bus,2004,"Akron, OH Metro Area",NaN,NaN,5.907162e+06,4.315393e+06,5.907162e+06,...,0,0,0,0,0,0,0,0,0,0
3,3,10420,Bus,2005,"Akron, OH Metro Area",NaN,NaN,6.013681e+06,4.053011e+06,6.013681e+06,...,0,0,0,0,0,0,0,0,0,0
4,4,10420,Bus,2006,"Akron, OH Metro Area",NaN,NaN,6.070797e+06,3.994786e+06,6.070797e+06,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4560,4560,49740,Bus,2014,"Yuma, AZ Metro Area",NaN,NaN,4.703299e+05,8.031786e+05,4.703299e+05,...,0,0,0,0,0,0,0,0,0,0
4561,4561,49740,Bus,2015,"Yuma, AZ Metro Area",NaN,NaN,4.394423e+05,8.329407e+05,4.394423e+05,...,0,0,0,0,0,0,0,0,0,0
4562,4562,49740,Bus,2016,"Yuma, AZ Metro Area",NaN,NaN,4.118686e+05,8.294332e+05,4.118686e+05,...,0,0,0,0,0,0,0,0,0,0
4563,4563,49740,Bus,2017,"Yuma, AZ Metro Area",NaN,NaN,4.176748e+05,8.357594e+05,4.176748e+05,...,0,0,0,0,0,0,0,0,0,0


In [6]:
import pandas as pd
Employee = [['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'],
        ['CSE', 'CSE', 'EEE', 'EEE', 'CE', 'CE', 'ME', 'ME'],
        ['Cat-1', 'Cat-2', 'Cat-1', 'Cat-2', 'Cat-1', 'Cat-2', 'Cat-1', 'Cat-2']]

index = pd.MultiIndex.from_arrays(Employee, names=['Name', 'Dept', 'Category'])

Scale = [1, 2, 2, 3, 3, 1, 2, 3]
Salary = [100, 200, 200, 300, 300, 100, 200, 300]

df = pd.DataFrame({'scale': Scale,
               'salary': Salary},
              index=index)

df

,,,scale,salary
Name,Dept,Category,,
A,CSE,Cat-1,1,100
B,CSE,Cat-2,2,200
C,EEE,Cat-1,2,200
D,EEE,Cat-2,3,300
E,CE,Cat-1,3,300
F,CE,Cat-2,1,100
G,ME,Cat-1,2,200
H,ME,Cat-2,3,300


In [5]:
df.groupby(['Category','scale']).sum().groupby('Category').cumsum()

salary
Category scale        
Cat-1    1         100
         2         500
         3         800
Cat-2    1         100
         2         300
         3         900